In [40]:
%%javascript
Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('up');

Jupyter.keyboard_manager.command_shortcuts.remove_shortcut('down');


<IPython.core.display.Javascript object>

In [41]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [42]:
# import glob

# print(glob.glob('*.csv'))

# final_df = pd.DataFrame()
# for i in ['0_9999k_data.csv','10k_19k_data.csv', '20k_29k_data.csv']:
#     one_part = pd.read_csv(i)
#     final_df = final_df.append(one_part)

# final_df.to_pickle("27k_data.pkl")
# final_df.to_csv("27k_data.csv", index=False)

In [43]:
final_df = pd.read_csv('27k_data.zip')
final_df = final_df.rename(columns = lambda x : str(x).replace(':',''))
final_df = final_df.rename(columns = lambda x : str(x).replace(" ",'_'))
final_df = final_df.apply(lambda x: x.replace(np.NaN,''))
final_df = final_df.apply(lambda x: x.astype(str).str.lower())
final_df.shape

(26811, 18)

## Dropping the duplicate movie rows

In [44]:
final_df = final_df.drop_duplicates(subset='movie_name')
final_df.shape

(9596, 18)

In [45]:
# 1. Tell that we scraped from 10K movies to 27K movies
# 	top_5_celeb_names	critic_line	critic_url	audience_review	quotes	Box_Office



In [46]:
def stop_words_removal(only_text):
    only_text = only_text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    return only_text

In [47]:
# rating, genre, directed by, release date ( In_Theaters), Runtime, Studio, Movie_name, top_5_celeb_name, rating_words

### Rating

In [48]:
final_df['Rating_words'] = final_df['Rating'].str.split("\s+\(").str[1]
final_df['Rating_words'] = final_df.Rating_words.astype(str)
final_df['Rating_words'] = final_df.Rating_words.str.replace('[^a-z \n]',"")
final_df['Rating_words'] = stop_words_removal(final_df.Rating_words)
final_df['Rating_words'] = final_df.Rating_words.str.replace('nan','')
# final_df['Rating_words'] = final_df.Rating_words.str.strip()
# final_df['Rating_words'] = final_df.Rating_words.str.replace(' ','_')

final_df['Rating'] = final_df['Rating'].str.split("\s+\(").str[0]
final_df['Rating'] = final_df.Rating.replace("nr"," ")
final_df['Rating'] = final_df.Rating.str.strip()

### Director

In [49]:
final_df['Directed_By'] = final_df.Directed_By.str.replace('[^a-z \n]',"")
final_df['Directed_By_1'] = final_df['Directed_By'].str.split(',').str[0]
final_df['Directed_By_1'] = final_df['Directed_By_1'].fillna(" ")
final_df['Directed_By_1'] = final_df.Directed_By_1.str.strip()
final_df['Directed_By_1'] = final_df.Directed_By_1.str.replace(' ','_')
# final_df['Directed_By_2'] = final_df['Directed_By'].str.split(',').str[1]
# # final_df['Directed_By_2'] = final_df['Directed_By_2'].fillna(" ")


### Release Year

In [50]:
final_df['release_year'] = final_df.In_Theaters.str.split(' ').str[2]
final_df['release_year'] = final_df.release_year.str.replace('[^0-9 \n]',"")

final_df['release_year'] = final_df.release_year.fillna(0)
final_df['release_year'] = final_df.release_year.astype(int)


final_df.loc[(final_df['release_year'] >=1920) & (final_df['release_year'] <1950), 'release_year_brackets'] ='black_white'
final_df.loc[(final_df['release_year'] >=1950) & (final_df['release_year'] <=1980), 'release_year_brackets'] ='pretty_old'
final_df.loc[(final_df['release_year'] >=1980) & (final_df['release_year'] <=1990), 'release_year_brackets'] ='year_80_90'
final_df.loc[(final_df['release_year'] >=1990) & (final_df['release_year'] <=2000), 'release_year_brackets'] ='year_90_2000'
final_df.loc[(final_df['release_year'] >=2000) & (final_df['release_year'] <=2010), 'release_year_brackets'] ='year_2000_2010'
final_df.loc[(final_df['release_year'] >=2010) & (final_df['release_year'] <=2020), 'release_year_brackets'] ='year_2010_2020'

final_df.loc[(final_df.release_year == 0), 'release_year_brackets'] = ''

### RunTime

In [51]:
final_df['Runtime'] = final_df.Runtime.str.split(" ").str[0]
final_df['Runtime'] = final_df.Runtime.replace("",np.NaN)
final_df['Runtime'] = final_df['Runtime'].fillna(0)
final_df['Runtime'] = final_df['Runtime'].astype(int)

In [52]:
final_df.loc[(final_df['Runtime'] > 10) & (final_df['Runtime'] < 90), 'runtime_brackets'] = 'short_till90'
final_df.loc[(final_df['Runtime'] >= 90) & (final_df['Runtime'] < 120), 'runtime_brackets'] = 'conventional_90_120'
final_df.loc[(final_df['Runtime'] >= 120) & (final_df['Runtime'] < 180), 'runtime_brackets'] = 'lengthy_120_180'
final_df.loc[(final_df['Runtime'] >= 180) & (final_df['Runtime'] < 800), 'runtime_brackets'] = 'super_lengthy_180_800'

final_df.loc[(final_df.runtime_brackets == 0), 'runtime_brackets'] = ' '

final_df['runtime_brackets'] = final_df.runtime_brackets.fillna(" ")


### Genre

In [53]:
final_df['Genre'] = stop_words_removal(final_df.Genre)
final_df['Genre'] = final_df.Genre.str.replace('[^a-z \n]',"")
final_df['Genre_1'] = final_df.Genre.str.split(' ').str[0] #+ ' ' + final_df.Genre.str.split(' ').str[1] 
final_df['Genre_2'] = final_df.Genre.str.split(' ').str[1] #+ ' ' + final_df.Genre.str.split(' ').str[1] 
final_df['Genre_2'] = final_df.Genre_2.fillna(" ")
final_df['Genre_3'] = final_df.Genre.str.split(' ').str[2] #+ ' ' + final_df.Genre.str.split(' ').str[1] 
final_df['Genre_3'] = final_df.Genre_3.fillna(" ")
final_df['Genre_cleaned'] = final_df.Genre_1 + ' ' + final_df.Genre_2 + ' ' +final_df.Genre_3
# final_df.Genre_cleaned + ' ' + final_df.Genre.str.split(' ').str[2]

In [54]:
final_df['Genre_cleaned'] = final_df.Genre_cleaned.str.strip()
final_df['Genre_cleaned'] =  final_df.Genre_cleaned.str.replace(' ','_')

### Celebrity Names

In [55]:
final_df['top_5_celeb_names'] = final_df.top_5_celeb_names.replace(",",' ')
final_df['top_5_celeb_names'] = final_df.top_5_celeb_names.str.replace('[^a-z \n]'," ")

### Critic Score

In [56]:
final_df['critic_score'] = final_df.critic_score.str.replace('%','')
final_df['critic_score'] = final_df.critic_score.astype(int)
final_df.loc[(final_df['critic_score'] >= 80) & (final_df['critic_score'] <= 100), 'critic_score_brackets'] = 'best_critic_movie'

final_df.loc[(final_df['critic_score'] >= 60) & (final_df['critic_score'] < 80), 'critic_score_brackets'] = 'good_critic_movie'
final_df.loc[(final_df['critic_score'] >= 40) & (final_df['critic_score'] < 60), 'critic_score_brackets'] = 'avg_critic_movie'
final_df.loc[(final_df['critic_score'] >= 20) & (final_df['critic_score'] < 40), 'critic_score_brackets'] = 'below_avg_critic_movie'
final_df.loc[(final_df['critic_score'] >= 0) & (final_df['critic_score'] < 20), 'critic_score_brackets'] = 'worst_critic_movie'


In [57]:
final_df.critic_score.isna().sum()

0

### Audience Score

In [58]:
final_df['audience_score'] = final_df.audience_score.str.replace('%','')
final_df['audience_score'] = final_df.audience_score.replace('','999')
final_df['audience_score'] = final_df.audience_score.astype(int)
final_df.loc[(final_df['audience_score'] >= 80) & (final_df['audience_score'] <= 100), 'audience_score_brackets'] = 'best_audience_movie'
final_df.loc[(final_df['audience_score'] >= 60) & (final_df['audience_score'] < 80), 'audience_score_brackets'] = 'good_audience_movie'
final_df.loc[(final_df['audience_score'] >= 40) & (final_df['audience_score'] < 60), 'audience_score_brackets'] = 'avg_audience_movie'
final_df.loc[(final_df['audience_score'] >= 20) & (final_df['audience_score'] < 40), 'audience_score_brackets'] = 'below_avg_audience_movie'
final_df.loc[(final_df['audience_score'] >= 0) & (final_df['audience_score'] < 20), 'audience_score_brackets'] = 'worst_audience_movie'

final_df['audience_score_brackets'] = final_df.audience_score_brackets.replace(999,'')
final_df['audience_score_brackets'] = final_df.audience_score_brackets.fillna('')


### Critic_consensus

In [59]:
final_df['critic_consensus'] = final_df.critic_consensus.str.replace('[^a-z \n]'," ")
final_df['critic_consensus'] = stop_words_removal(final_df.critic_consensus)
final_df['critic_consensus'] = final_df.critic_consensus.str.strip()

## Studio

In [60]:
final_df['Studio'] = final_df.Studio.str.replace('[^a-z \n]'," ")
final_df['Studio'] = stop_words_removal(final_df.Studio)
final_df['Studio'] = final_df.Studio.str.replace(' ','_')

In [63]:
cosine_df = final_df[['Rating','Genre_cleaned','Directed_By_1',
                      'runtime_brackets', 'Studio','movie_name','critic_consensus',
                     'critic_score_brackets','audience_score_brackets','top_5_celeb_names','Rating_words',
                      'release_year_brackets']]

In [65]:
cosine_df['combined'] = cosine_df[cosine_df.columns].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)

<ipython-input-65-0262324e13d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosine_df['combined'] = cosine_df[cosine_df.columns].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)


In [74]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

In [71]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(cosine_df['combined'])

In [72]:
tfidf_matrix

<9596x213545 sparse matrix of type '<class 'numpy.float64'>'
	with 524064 stored elements in Compressed Sparse Row format>

In [75]:
# http://scikit-learn.org/stable/modules/metrics.html#linear-kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [76]:
cosine_sim[0]

array([1.        , 0.01016072, 0.        , ..., 0.        , 0.        ,
       0.00235269])

In [78]:
cosine_df = cosine_df.reset_index(drop=True)
titles = cosine_df['movie_name']
indices = pd.Series(cosine_df.index, index=cosine_df['movie_name'])

In [79]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [95]:
indices.index[444]

'love aaj kal'

In [100]:
indices.index[1122]

'spider-man: into the spider-verse'

In [99]:
get_recommendations('i hate new york').head(10)

3709                  banksy does new york
411     pete davidson: alive from new york
1499               susanne bartsch: on top
6942                    sunday in new york
7638                  new york, i love you
7214                  ny export: opus jazz
6675              bill cunningham new york
5889                    2 days in new york
3699                   welcome to new york
1213                       the hate u give
Name: movie_name, dtype: object

In [102]:
get_recommendations('love aaj kal').head(10)

6200                   jannat 2
280                  extraction
4200                agent vinod
8460                     tashan
9290              ta ra rum pum
6322                   rockstar
4320                   i am ali
6048                   cocktail
4264                  aarakshan
8492    thoda pyaar thoda magic
Name: movie_name, dtype: object

In [101]:
get_recommendations('spider-man: into the spider-verse').head(10)

5869          the amazing spider-man
9075                    spider-man 3
702        spider-man: far from home
2172          spider-man: homecoming
1182    the girl in the spider's web
4504        the amazing spider-man 2
1274                           venom
3584                            dope
8464        kiss of the spider woman
5015                 big ass spider!
Name: movie_name, dtype: object